In [1]:
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score as auc

In [2]:
from mriqc_learn.datasets import load_dataset
from mriqc_learn.models import preprocess as pp
from mriqc_learn.models.production import init_pipeline
from mriqc_learn.model_selection import split

## Load some data
We first load the ABIDE dataset, one of the default datasets distributed with MRIQC-learn

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_excel('/home/jaimebarranco/Desktop/MRI-QC/scores.xlsx', sheet_name='brainmask_avg_data')
df['site'] = 'SHIP'

# Separate features (X) and target (y)
X = df.drop(columns=['name','sub','subject','rating','rating_text','blur','blur_text','noise','noise_text','motion','motion_text','bgair','bgair_text','exclusion'])
y = df[['rating','site']]

features = X.columns.tolist()

# Train/test split
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
train_x = train_x.drop(columns=[
    "size_x",
    "size_y",
    "size_z",
    "spacing_x",
    "spacing_y",
    "spacing_z",
    # "summary_bg_p05", # all zeros
])
numeric_columns = train_x.columns.tolist()

In [9]:
train_y = train_y[["rating"]].values.squeeze().astype(int)
print(f'Excluded={100 * (train_y < 1).sum() / len(train_y)}')
print(f'Accept={100 * (train_y >= 1).sum() / len(train_y)}')
train_y[train_y >= 1] = 1

Excluded=12.121212121212121
Accept=87.87878787878788


Let's pick the ratings from "rater_3" and binarize the three categories into only two.
We can also see that the dataset is unbalanced.

Let's print out a pretty view of the data table:

In [5]:
train_x

,cjv,cnr,efc,fber,fwhm_avg,fwhm_x,fwhm_y,fwhm_z,icvs_csf,icvs_gm,...,summary_wm_median,summary_wm_n,summary_wm_p05,summary_wm_p95,summary_wm_stdv,tpm_overlap_csf,tpm_overlap_gm,tpm_overlap_wm,wm2max,site
57,0.448414,3.114049,0.597335,37197.929282,3.604710,3.46635,3.90859,3.43919,0.169228,0.360908,...,1000.083417,352123,933.288886,1083.555735,45.994151,0.169087,0.422903,0.515549,0.603553,SHIP
80,0.510556,2.655019,0.581888,18742.146867,3.547583,3.44738,3.74877,3.44660,0.218436,0.338062,...,1000.104941,207191,924.826720,1105.795424,56.239391,0.175538,0.409606,0.501502,0.485838,SHIP
73,0.503120,2.746299,0.573094,20870.499605,3.566697,3.53848,3.75553,3.40608,0.281562,0.327149,...,1000.094484,177628,920.598422,1103.779523,56.302368,0.158704,0.368002,0.451359,0.512418,SHIP
43,0.406657,3.377574,0.539746,31178.174972,3.571273,3.47532,3.81148,3.42702,0.220583,0.376211,...,1000.083453,206828,936.484522,1077.954585,43.285126,0.169976,0.443589,0.516135,0.601794,SHIP
60,0.387697,3.587771,0.522215,26844.481837,3.546927,3.35994,3.84124,3.43960,0.191132,0.402675,...,1000.059080,173073,933.443594,1080.936658,45.078082,0.158113,0.448850,0.512629,0.579524,SHIP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,0.516005,2.664041,0.598902,14380.006944,3.694067,3.52521,4.04490,3.51209,0.295019,0.314556,...,1000.111644,161754,913.470519,1114.541916,61.525191,0.135471,0.333124,0.398176,0.471318,SHIP
67,0.448091,3.094118,0.543796,28555.917519,3.500230,3.33013,3.76717,3.40339,0.215502,0.381438,...,1000.062877,175130,930.591960,1088.987413,48.578024,0.170574,0.444431,0.521019,0.533905,SHIP
64,0.445249,3.163178,0.580320,-1.000000,3.503080,3.34008,3.73391,3.43525,0.205994,0.381840,...,1000.086505,228421,929.916123,1089.111214,48.832193,0.187022,0.451585,0.519289,0.575023,SHIP
47,0.445324,3.058757,0.554685,30115.786983,3.620553,3.49549,3.85356,3.51261,0.221696,0.372123,...,1000.055751,230176,936.090105,1088.865944,46.824035,0.172665,0.427342,0.510288,0.613959,SHIP


## Cross-validation of the default classifier
Let's cross-validate the performance of our classifier using a Leave-one-site-out strategy.

We can now feed the model into the cross-validation loop:

In [16]:
cv_score = cross_val_score(
    init_pipeline(),
    X=train_x,
    y=train_y,
    cv=5,
    scoring="roc_auc",
    n_jobs=16,
    # error_score="raise",
)

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/home/jaimebarranco/miniconda3/envs/a-eye/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/jaimebarranco/miniconda3/envs/a-eye/lib/python3.8/site-packages/sklearn/pipeline.py", line 378, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/jaimebarranco/miniconda3/envs/a-eye/lib/python3.8/site-packages/sklearn/pipeline.py", line 336, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/home/jaimebarranco/miniconda3/envs/a-eye/lib/python3.8/site-packages/joblib/memory.py", line 349, in __call__

After one or two minutes, the scores have been caculated for each of the 14 folds our splitter created.
The average performance is AUC=0.885.

In [ ]:
print(cv_score)
cv_score.mean()

In [ ]:
custom_cv_score = {}
for train_index, (site, test_index) in outer_cv.split(train_x, y=train_y, return_key=True):
    # Validate on test fold
    print(f"Validating on left-out site ({site})...")
    model_split = init_pipeline()
    model_split = model_split.fit(train_x.iloc[train_index], train_y[train_index])
    custom_cv_score[site] = auc(train_y[test_index], model_split.predict(train_x.iloc[test_index]))

In [ ]:
print(custom_cv_score)
np.mean(list(custom_cv_score.values()))

We now train the model on all available training data:

In [ ]:
model = init_pipeline().fit(
    X=train_x,
    y=train_y,
)    

We can easily see the effects of overfitting by evaluating the classifier on the same folds we used for cross-validation.

In [ ]:
overfit_cv_score = {}
for train_index, (site, test_index) in outer_cv.split(train_x, y=train_y, return_key=True):
    print(f"Validating on left-out site ({site})...")
    overfit_cv_score[site] = auc(train_y[test_index], model.predict(train_x.iloc[test_index]))

In [ ]:
print([overfit_cv_score[s] - custom_cv_score[s] for s in overfit_cv_score.keys()])

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(train_y, model.predict(train_x)))

## Evaluating on held-out dataset
We first load the held-out dataset in, and evaluate:

In [ ]:
(test_x, test_y), (_, _) = load_dataset("ds030", split_strategy="none")
test_x["site"] = test_y.site
test_x

In [ ]:
has_ghost = test_y.has_ghost.values.astype(bool)
test_y = test_y[["rater_1"]].values.squeeze().astype(int)
print(f"Discard={100 * (test_y == -1).sum() / len(test_y)}")
print(f"Doubtful={100 * (test_y == 0).sum() / len(test_y)}")
print(f"Accept={100 * (test_y == 1).sum() / len(test_y)}")
test_y[test_y < 1] = 0

In [ ]:
auc(test_y, model.predict(test_x))

In [ ]:
auc(test_y[~has_ghost], model.predict(test_x[~has_ghost]))

In [ ]:
print(classification_report(test_y, model.predict(test_x)))

In [ ]:
print(classification_report(test_y[~has_ghost], model.predict(test_x[~has_ghost])))

## Nested cross-validation

In [ ]:
p_grid = [{
    "scale__unit_variance": [True, False],
    "scale__with_centering": [True, False],
    "site_pred__disable": [False, True],
    "winnow__disable": [False, True],
    "svc__kernel": ["rbf"],
    "svc__C": [10],
    "svc__gamma": [0.1],
}]

In [ ]:
# Nested CV with parameter optimization
inner_cv = split.LeavePSitesOut(1, robust=True)
inner_cv.get_n_splits(X=train_x, y=train_y)

clf = GridSearchCV(
    estimator=pipe,
    param_grid=p_grid,
    cv=inner_cv,
    n_jobs=30,
    scoring="roc_auc",
)
# clf.fit(train_x, y=train_y)

In [ ]:
nested_score = cross_val_score(
    clf,
    X=train_x,
    y=train_y,
    cv=outer_cv,
    scoring="roc_auc",
    verbose=10,
    n_jobs=16,
)
nested_score.mean()

In [ ]:
clf.cv_results_

In [ ]:
clf.best_params_